<a href="https://colab.research.google.com/github/IyadSultan/AI_pediatric_oncology/blob/main/05_prompting_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🩺 Open in Colab → Prompting the OpenAI API — Better, Faster, Safer

A 40-minute, cut-to-the-chase notebook for beginner-to-intermediate users.

---

## ⏱️ Agenda

| Section | Time | What you’ll master |
|:-------:|:----:|:------------------:|
| 1 | 3 min | Setup & first call |
| 2 | 5 min | Counting tokens (why cost ≠ mystery) |
| 3 | 18 min | Six prompting techniques |
| 4 | 6 min | Tuning parameters (temperature, top-p, etc.) |
| 5 | 6 min | Debugging & auto-fixing bad JSON |
| 6 | 2 min | Next-level ideas & homework |

*(Copy each cell verbatim into Colab → run top-to-bottom.)*

---

## 1 | Setup & “Hello GPT-4o” (3 min)

**Install packages:**

In [2]:
!pip install -q openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00


**Import libraries and set up API key:**

In [3]:
import os, getpass, openai
os.environ["OPENAI_API_KEY"] = getpass.getpass("🔑 Paste your OpenAI key:")
client = openai.OpenAI()   # key auto-read from env

🔑 Paste your OpenAI key:··········


In [9]:
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[{"role":"user","content":"Give me a 1-sentence health tip."}]
)
print(resp.choices[0].message.content)

Stay hydrated by drinking plenty of water throughout the day to support overall health and well-being.


## 2 | Tokens = Time = Money (5 min)

**Install `tiktoken` and explore tokenization:**

In [10]:
!pip install -q tiktoken
import tiktoken, textwrap

enc   = tiktoken.get_encoding("cl100k_base")
utter = "I go to work every morning at 8 a.m. and get back at 5 p.m."
ids   = enc.encode(utter)
print(ids)                  # raw token ids
print("→", len(ids), "tokens")

[40, 733, 311, 990, 1475, 6693, 520, 220, 23, 264, 749, 13, 323, 636, 1203, 520, 220, 20, 281, 749, 13]
→ 21 tokens



> **Rule of thumb:**  
> 1 token ≈ ¾ English word.

> **Tip:**  
> Keep prompts short & reuse variables to save tokens → save money.

## 3 | Six Prompting Styles (18 min)

### 3.1 Zero-shot (prompt ≈ instruction)

In [11]:
prompt = "Translate to French: 'How are you today?'"
msg    = [{"role":"user","content": prompt}]
print(client.chat.completions.create(model="gpt-4o-mini", messages=msg)
      .choices[0].message.content)

The translation of "How are you today?" in French is "Comment ça va aujourd'hui ?" or simply "Ça va aujourd'hui ?" for a more casual tone.


### 3.2 Few-shot (examples → clarity)


In [12]:
fewshot = """
Translate EN → FR
1) Hello, how are you? -> Bonjour, comment ça va ?
2) What is your name?  -> Comment vous appelez-vous ?
3) I love programming. -> J'aime programmer.
4) Have a great day.   ->
"""
print(client.chat.completions.create(model="gpt-4o-mini",
      messages=[{"role":"user","content":fewshot}]).choices[0].message.content)

4) Have a great day. -> Passez une excellente journée.


### 3.3 Chain-of-Thought (think step-by-step)

In [13]:
cot = """Solve step-by-step:
A train goes 60 mph. How far in 3 h?
#### Reasoning:"""
print(client.chat.completions.create(model="gpt-4o-mini",
      messages=[{"role":"user","content":cot}]).choices[0].message.content)

To determine how far a train travels in a given period of time, you can use the formula for distance:

\[
\text{Distance} = \text{Speed} \times \text{Time}
\]

### Step 1: Identify the variables
- Speed of the train = 60 mph (miles per hour)
- Time = 3 hours

### Step 2: Substitute the values into the formula
Using the formula we identified, we substitute the speed and time:

\[
\text{Distance} = 60 \text{ mph} \times 3 \text{ h}
\]

### Step 3: Perform the multiplication
Now, we multiply 60 by 3:

\[
\text{Distance} = 180 \text{ miles}
\]

### Conclusion
The train travels **180 miles** in 3 hours.



### 3.4 Instruction-based (clarity first)

In [14]:
instr = ("You are a helpful assistant. Rewrite professionally:\n"
         "'Can you send me the details ASAP?'")
print(client.chat.completions.create(model="gpt-4o-mini",
      messages=[{"role":"user","content":instr}]).choices[0].message.content)

Could you please send me the details at your earliest convenience? Thank you.


### 3.5 Role-play (domain voice)

In [15]:
role_prompt = "Suggest a 3-day low-impact workout plan."
print(client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role":"system","content":"You are a certified physiotherapist."},
                {"role":"user","content":role_prompt}]
).choices[0].message.content)

Here's a 3-day low-impact workout plan that focuses on improving strength, flexibility, and cardiovascular fitness. Each session includes a warm-up, a main workout, and a cooldown/stretch segment.

### Day 1: Full Body Strength & Mobility

**Warm-Up (5-10 minutes)**
- Arm circles (1 minute)
- Leg swings (front to back and side to side, 1 minute each leg)
- Gentle torso twists (1 minute)

**Main Workout (30-40 minutes)**  
Repeat 2-3 times, resting 60 seconds between sets.
1. **Bodyweight Squats** - 10-15 reps
2. **Wall Push-Ups** - 8-12 reps
3. **Seated Leg Raises** - 10-15 reps each leg
4. **Glute Bridges** - 10-15 reps
5. **Seated Rows with Resistance Band** - 10-15 reps
6. **Standing Heel Raises** - 10-15 reps

**Cooldown/Stretch (5-10 minutes)**
- Child’s pose (30 seconds)
- Seated forward bend (30 seconds)
- Standing quad stretch (30 seconds each leg)
- Gentle neck stretches (side-to-side holding each for 15-30 seconds)

---

### Day 2: Cardiovascular & Core Stability

**Warm-Up (

### 3.6 Few-shot + context (best of both)

In [16]:
combo = ("You are a language expert. Translate casually to Spanish:\n"
         "1) Where are you going? -> ¿Adónde vas?\n"
         "2) I'm just hanging out. -> Solo estoy pasando el rato.\n"
         "3) Let me know when you're free. ->")
print(client.chat.completions.create(model="gpt-4o-mini",
      messages=[{"role":"user","content":combo}],
      temperature=0.7, max_tokens=10).choices[0].message.content)

Déjame saber cuando estés libre.


## 4 | Fine-tuning Parameters (6 min)



| Param | Why it matters |
|:-----:|:--------------:|
| temperature | 0 = deterministic, 1 = creative |
| top_p | nucleus sampling (diversity) |
| max_tokens | output length cap |
| frequency / presence penalties | reduce repetition, encourage novelty |

> **Mini-lab:**  
> Re-run the translator with `temperature=1.0` and `top_p=0.3`.  
> Collect 3 outputs — notice variability.

## 5 | Debugging & Auto-Fixing JSON (6 min)

### 5.1 Ask for JSON

In [ ]:
ask = ("Return valid JSON with key 'translation'.\n"
       "Text: Hello!")
resp = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role":"user","content":ask}])
print(resp.choices[0].message.content)

**Sometimes the model fails (extra commentary). Let’s validate and auto-fix:**

In [ ]:
import json, langchain.output_parsers.fix as fix
from langchain_core.output_parsers import JsonOutputParser

parser  = JsonOutputParser()
fixer   = fix.OutputFixingParser.from_llm(parser=parser, llm=None)  # self-heal via GPT

def safe_parse(txt):
    try:
        return parser.parse(txt)
    except Exception:
        return fixer.parse(txt)

print(safe_parse(resp.choices[0].message.content))

## 6 | Next-Level Ideas & Homework (2 min)

🚀 **Where to go from here:**

- **LangChain** for robust apps (prompt templates, memory, RAG).
- **LangGraph** for multi-step triage or care-path flows.
- **LangSmith** for logging, eval & prompt A/B testing.

📝 **Homework:**

Using the melanoma case note below, build a PubMed search tool (e.g., using E-utilities API) that fetches the five newest papers on late-line melanoma therapies and asks GPT-4o to summarize which option fits the patient best.